In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import linearmodels as lm

from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS

In [ ]:
c = pd.read_csv("NYPD_Complaint_Data_Historic.csv") ## choose "panel-for-R.csv" and more information on variables are here: http://sda.berkeley.edu/sdaweb/analysis/?dataset=gss06panelw3 ##

c.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,522575447,08/29/2006,13:00:00,NaN,NaN,43.0,08/30/2006,578,HARRASSMENT 2,638.0,...,M,NaN,40.827414,-73.877946,"(40.827414051, -73.877945775)",PATROL BORO BRONX,NaN,25-44,BLACK HISPANIC,F
1,403507361,11/05/2006,11:00:00,11/05/2006,17:40:00,66.0,11/05/2006,107,BURGLARY,221.0,...,NaN,NaN,40.637097,-74.006105,"(40.637096864, -74.006105014)",PATROL BORO BKLYN SOUTH,NaN,45-64,ASIAN / PACIFIC ISLANDER,F
2,631420068,09/08/2006,23:30:00,09/09/2006,00:01:00,106.0,09/09/2006,347,INTOXICATED & IMPAIRED DRIVING,905.0,...,NaN,NaN,40.679260,-73.841500,"(40.679260229, -73.841500185)",PATROL BORO QUEENS SOUTH,NaN,NaN,UNKNOWN,E
3,995609899,12/13/2011,18:40:00,12/13/2011,18:49:00,79.0,12/13/2011,341,PETIT LARCENY,333.0,...,NaN,NaN,40.687402,-73.940369,"(40.687401619, -73.940369194)",PATROL BORO BKLYN NORTH,NaN,NaN,UNKNOWN,D
4,480667624,08/14/2009,04:20:00,NaN,NaN,30.0,08/14/2009,113,FORGERY,729.0,...,NaN,NaN,40.831576,-73.942983,"(40.831576128, -73.942982699)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E


In [ ]:
c.drop(['KY_CD', 'PD_CD', 'PD_DESC', 'HADEVELOPT', 'PARKS_NM', 'JURIS_DESC', 'HOUSING_PSA'], axis=1)
# list(c)
c.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,ADDR_PCT_CD,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,522575447,08/29/2006,13:00:00,NaN,NaN,43.0,08/30/2006,578,HARRASSMENT 2,638.0,...,M,NaN,40.827414,-73.877946,"(40.827414051, -73.877945775)",PATROL BORO BRONX,NaN,25-44,BLACK HISPANIC,F
1,403507361,11/05/2006,11:00:00,11/05/2006,17:40:00,66.0,11/05/2006,107,BURGLARY,221.0,...,NaN,NaN,40.637097,-74.006105,"(40.637096864, -74.006105014)",PATROL BORO BKLYN SOUTH,NaN,45-64,ASIAN / PACIFIC ISLANDER,F
2,631420068,09/08/2006,23:30:00,09/09/2006,00:01:00,106.0,09/09/2006,347,INTOXICATED & IMPAIRED DRIVING,905.0,...,NaN,NaN,40.679260,-73.841500,"(40.679260229, -73.841500185)",PATROL BORO QUEENS SOUTH,NaN,NaN,UNKNOWN,E
3,995609899,12/13/2011,18:40:00,12/13/2011,18:49:00,79.0,12/13/2011,341,PETIT LARCENY,333.0,...,NaN,NaN,40.687402,-73.940369,"(40.687401619, -73.940369194)",PATROL BORO BKLYN NORTH,NaN,NaN,UNKNOWN,D
4,480667624,08/14/2009,04:20:00,NaN,NaN,30.0,08/14/2009,113,FORGERY,729.0,...,NaN,NaN,40.831576,-73.942983,"(40.831576128, -73.942982699)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E


In [ ]:
c.rename(columns={"CMPLNT_NUM":"complaint_number", "CMPLNT_FR_DT":"complaint_from_date", 
          "CMPLNT_FR_TM":"complaint_from_time", "CMPLNT_TO_DT": "complaint_to_date", 
          "CMPLNT_TO_TM":"complaint_to_time", "OFNS_DESC":"offense_description", "BORO_NM":"borough"
                  })

,complaint_number,complaint_from_date,complaint_from_time,complaint_to_date,complaint_to_time,ADDR_PCT_CD,RPT_DT,KY_CD,offense_description,PD_CD,...,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX
0,522575447,08/29/2006,13:00:00,NaN,NaN,43.0,08/30/2006,578,HARRASSMENT 2,638.0,...,M,NaN,40.827414,-73.877946,"(40.827414051, -73.877945775)",PATROL BORO BRONX,NaN,25-44,BLACK HISPANIC,F
1,403507361,11/05/2006,11:00:00,11/05/2006,17:40:00,66.0,11/05/2006,107,BURGLARY,221.0,...,NaN,NaN,40.637097,-74.006105,"(40.637096864, -74.006105014)",PATROL BORO BKLYN SOUTH,NaN,45-64,ASIAN / PACIFIC ISLANDER,F
2,631420068,09/08/2006,23:30:00,09/09/2006,00:01:00,106.0,09/09/2006,347,INTOXICATED & IMPAIRED DRIVING,905.0,...,NaN,NaN,40.679260,-73.841500,"(40.679260229, -73.841500185)",PATROL BORO QUEENS SOUTH,NaN,NaN,UNKNOWN,E
3,995609899,12/13/2011,18:40:00,12/13/2011,18:49:00,79.0,12/13/2011,341,PETIT LARCENY,333.0,...,NaN,NaN,40.687402,-73.940369,"(40.687401619, -73.940369194)",PATROL BORO BKLYN NORTH,NaN,NaN,UNKNOWN,D
4,480667624,08/14/2009,04:20:00,NaN,NaN,30.0,08/14/2009,113,FORGERY,729.0,...,NaN,NaN,40.831576,-73.942983,"(40.831576128, -73.942982699)",PATROL BORO MAN NORTH,NaN,NaN,UNKNOWN,E
5,605915885,10/27/2009,21:10:00,NaN,NaN,69.0,10/29/2009,105,ROBBERY,397.0,...,M,NaN,40.643368,-73.905059,"(40.643368102, -73.905059257)",PATROL BORO BKLYN SOUTH,NaN,18-24,BLACK,M
6,820084484,12/16/2006,20:00:00,12/16/2006,20:05:00,40.0,12/16/2006,235,DANGEROUS DRUGS,567.0,...,NaN,NaN,40.819127,-73.917392,"(40.819127068, -73.917392286)",PATROL BORO BRONX,NaN,NaN,UNKNOWN,E
7,799044409,07/21/2007,23:15:00,07/21/2007,23:45:00,75.0,07/22/2007,109,GRAND LARCENY,421.0,...,NaN,NaN,40.673329,-73.877262,"(40.673328553, -73.877261612)",PATROL BORO BKLYN NORTH,NaN,25-44,BLACK,M
8,211477358,07/07/2008,21:30:00,07/08/2008,11:45:00,34.0,07/08/2008,341,PETIT LARCENY,321.0,...,NaN,NaN,40.849415,-73.939425,"(40.849414928, -73.939425145)",PATROL BORO MAN NORTH,NaN,25-44,WHITE,M
9,986887005,09/17/2007,16:00:00,NaN,NaN,48.0,09/19/2007,578,HARRASSMENT 2,637.0,...,M,NaN,40.861392,-73.889980,"(40.861391939, -73.889979903)",PATROL BORO BRONX,NaN,<18,BLACK,M


In [ ]:
cf = c[["CMPLNT_NUM", "CMPLNT_FR_DT", 
          "CMPLNT_FR_TM", "CMPLNT_TO_DT", "LAW_CAT_CD",
          "CMPLNT_TO_TM", "OFNS_DESC", "BORO_NM",
        "SUSP_SEX", "SUSP_RACE", "SUSP_AGE_GROUP",
        "VIC_SEX", "VIC_RACE", "VIC_AGE_GROUP"
       ]]
cf.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,LAW_CAT_CD,CMPLNT_TO_TM,OFNS_DESC,BORO_NM,SUSP_SEX,SUSP_RACE,SUSP_AGE_GROUP,VIC_SEX,VIC_RACE,VIC_AGE_GROUP
0,522575447,08/29/2006,13:00:00,NaN,VIOLATION,NaN,HARRASSMENT 2,BRONX,M,UNKNOWN,NaN,F,BLACK HISPANIC,25-44
1,403507361,11/05/2006,11:00:00,11/05/2006,FELONY,17:40:00,BURGLARY,BROOKLYN,NaN,NaN,NaN,F,ASIAN / PACIFIC ISLANDER,45-64
2,631420068,09/08/2006,23:30:00,09/09/2006,MISDEMEANOR,00:01:00,INTOXICATED & IMPAIRED DRIVING,QUEENS,NaN,NaN,NaN,E,UNKNOWN,NaN
3,995609899,12/13/2011,18:40:00,12/13/2011,MISDEMEANOR,18:49:00,PETIT LARCENY,BROOKLYN,NaN,NaN,NaN,D,UNKNOWN,NaN
4,480667624,08/14/2009,04:20:00,NaN,FELONY,NaN,FORGERY,MANHATTAN,NaN,NaN,NaN,E,UNKNOWN,NaN


In [ ]:
cf.loc[cf['LAW_CAT_CD'] == 'FELONY']

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,LAW_CAT_CD,CMPLNT_TO_TM,OFNS_DESC,BORO_NM,SUSP_SEX,SUSP_RACE,SUSP_AGE_GROUP,VIC_SEX,VIC_RACE,VIC_AGE_GROUP
1,403507361,11/05/2006,11:00:00,11/05/2006,FELONY,17:40:00,BURGLARY,BROOKLYN,NaN,NaN,NaN,F,ASIAN / PACIFIC ISLANDER,45-64
4,480667624,08/14/2009,04:20:00,NaN,FELONY,NaN,FORGERY,MANHATTAN,NaN,NaN,NaN,E,UNKNOWN,NaN
5,605915885,10/27/2009,21:10:00,NaN,FELONY,NaN,ROBBERY,BROOKLYN,M,BLACK,NaN,M,BLACK,18-24
7,799044409,07/21/2007,23:15:00,07/21/2007,FELONY,23:45:00,GRAND LARCENY,BROOKLYN,NaN,NaN,NaN,M,BLACK,25-44
14,103912654,08/30/2009,01:00:00,08/30/2009,FELONY,01:01:00,FELONY ASSAULT,QUEENS,NaN,NaN,NaN,M,WHITE,25-44
15,591753685,12/09/2008,15:10:00,12/09/2008,FELONY,15:14:00,ROBBERY,QUEENS,M,WHITE HISPANIC,18-24,F,WHITE HISPANIC,45-64
18,935932450,04/11/2007,17:00:00,04/14/2007,FELONY,18:00:00,GRAND LARCENY,MANHATTAN,M,WHITE,25-44,M,WHITE,45-64
27,836522816,05/14/2008,23:00:00,05/14/2008,FELONY,23:10:00,BURGLARY,BRONX,NaN,NaN,NaN,D,UNKNOWN,NaN
34,320669846,07/27/2008,13:00:00,NaN,FELONY,NaN,MISCELLANEOUS PENAL LAW,BROOKLYN,M,BLACK HISPANIC,NaN,F,BLACK,25-44
36,718836666,08/26/2010,00:01:00,NaN,FELONY,NaN,GRAND LARCENY,BRONX,NaN,NaN,NaN,D,UNKNOWN,NaN


In [ ]:
pd.crosstab(index=cf['SUSP_SEX'], columns='count')

col_0,count
SUSP_SEX,
F,642101
M,1995894
U,507139


In [ ]:
pd.crosstab(index=cf['SUSP_RACE'], columns='count')

col_0,count
SUSP_RACE,
AMERICAN INDIAN/ALASKAN NATIVE,10195
ASIAN / PACIFIC ISLANDER,102247
BLACK,1224987
BLACK HISPANIC,168273
OTHER,11
UNKNOWN,847942
WHITE,366466
WHITE HISPANIC,558327


In [ ]:
pd.crosstab(index=complaint['BORO_NM'], columns='count')

col_0,count
BORO_NM,
BRONX,1408361
BROOKLYN,1935028
MANHATTAN,1557920
QUEENS,1282057
STATEN ISLAND,306986


In [ ]:
pd.crosstab(index=cf['VIC_RACE'], columns='count')

col_0,count
VIC_RACE,
AMERICAN INDIAN/ALASKAN NATIVE,29422
ASIAN / PACIFIC ISLANDER,360047
BLACK,1544439
BLACK HISPANIC,214165
OTHER,31
UNKNOWN,2186733
WHITE,1129716
WHITE HISPANIC,1036012


In [ ]:
pd.crosstab(index=cf['VIC_SEX'], columns='count')

col_0,count
VIC_SEX,
D,772964
E,1067365
F,2539404
M,2120829
U,4


In [ ]:
cf = cf.loc[cf['BORO_NM'] == ('BROOKLYN')] 
cf.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,LAW_CAT_CD,CMPLNT_TO_TM,OFNS_DESC,BORO_NM,SUSP_SEX,SUSP_RACE,SUSP_AGE_GROUP,VIC_SEX,VIC_RACE,VIC_AGE_GROUP
1,403507361,11/05/2006,11:00:00,11/05/2006,FELONY,17:40:00,BURGLARY,BROOKLYN,NaN,NaN,NaN,F,ASIAN / PACIFIC ISLANDER,45-64
3,995609899,12/13/2011,18:40:00,12/13/2011,MISDEMEANOR,18:49:00,PETIT LARCENY,BROOKLYN,NaN,NaN,NaN,D,UNKNOWN,NaN
5,605915885,10/27/2009,21:10:00,NaN,FELONY,NaN,ROBBERY,BROOKLYN,M,BLACK,NaN,M,BLACK,18-24
7,799044409,07/21/2007,23:15:00,07/21/2007,FELONY,23:45:00,GRAND LARCENY,BROOKLYN,NaN,NaN,NaN,M,BLACK,25-44
10,207970722,01/06/2007,16:00:00,02/26/2007,MISDEMEANOR,12:00:00,FRAUDS,BROOKLYN,NaN,UNKNOWN,NaN,M,WHITE,45-64


<h1><b>General Statistics</b></h1>


In [ ]:
conditions = [
    (cf['LAW_CAT_CD'] == 'VIOLATION') ,
    (cf['LAW_CAT_CD'] == 'MISDEMEANOR'),
    (cf['LAW_CAT_CD'] == 'FELONY')
]
choices = [1, 2, 3]
cf['LAW_CAT_CD'] = np.select(conditions, choices, default=np.nan)

pd.crosstab(index=cf['LAW_CAT_CD'], columns='count')

col_0,count
LAW_CAT_CD,
1.0,246489
2.0,1061916
3.0,626623


In [ ]:
# time = float(t_temp.replace(":",""))
def removeColon(x):
    if (isinstance(x, str)):
        hour = int(x[:2])
    #     print (hour)
        if (5 <= hour < 7 ):
            return 1
        elif (7 <= hour < 9):
            return 2
        elif (9 <= hour < 12):
            return 3
        elif (12 <= hour < 15):
            return 4
        elif (15 <= hour < 18):
            return 5
        elif (18 <= hour < 20):
            return 6
        elif (20 <= hour < 22):
            return 7
        elif ((22 <= hour <= 23) or hour == 0):
            return 8
        elif (0 < hour < 5): 
            return 9
        else:
            return None
    else:
        return None

cf['CMPLNT_FR_TM'] = cf['CMPLNT_FR_TM'].apply(removeColon)
cf

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,LAW_CAT_CD,CMPLNT_TO_TM,OFNS_DESC,BORO_NM,SUSP_SEX,SUSP_RACE,SUSP_AGE_GROUP,VIC_SEX,VIC_RACE,VIC_AGE_GROUP
1,403507361,11/05/2006,3.0,11/05/2006,3.0,17:40:00,BURGLARY,BROOKLYN,NaN,NaN,NaN,F,ASIAN / PACIFIC ISLANDER,45-64
3,995609899,12/13/2011,6.0,12/13/2011,2.0,18:49:00,PETIT LARCENY,BROOKLYN,NaN,NaN,NaN,D,UNKNOWN,NaN
5,605915885,10/27/2009,7.0,NaN,3.0,NaN,ROBBERY,BROOKLYN,M,BLACK,NaN,M,BLACK,18-24
7,799044409,07/21/2007,8.0,07/21/2007,3.0,23:45:00,GRAND LARCENY,BROOKLYN,NaN,NaN,NaN,M,BLACK,25-44
10,207970722,01/06/2007,5.0,02/26/2007,2.0,12:00:00,FRAUDS,BROOKLYN,NaN,UNKNOWN,NaN,M,WHITE,45-64
12,924188035,01/08/2010,8.0,NaN,2.0,NaN,ASSAULT 3 & RELATED OFFENSES,BROOKLYN,M,BLACK,NaN,F,BLACK,18-24
13,321158154,11/02/2007,3.0,11/02/2007,2.0,11:56:00,DANGEROUS DRUGS,BROOKLYN,NaN,NaN,NaN,E,UNKNOWN,NaN
19,600460494,11/25/2006,3.0,NaN,2.0,NaN,OFF. AGNST PUB ORD SENSBLTY &,BROOKLYN,F,UNKNOWN,NaN,M,WHITE,25-44
21,108113137,06/30/2009,5.0,06/30/2009,1.0,17:20:00,HARRASSMENT 2,BROOKLYN,F,WHITE HISPANIC,NaN,M,WHITE HISPANIC,25-44
23,314547311,11/24/2006,8.0,11/25/2006,2.0,12:00:00,PETIT LARCENY,BROOKLYN,NaN,NaN,NaN,M,WHITE,25-44


In [ ]:
cf[['CMPLNT_FR_TM', 'LAW_CAT_CD', 'OFNS_DESC', 'SUSP_SEX', 'SUSP_RACE', 'VIC_SEX', 'VIC_RACE']].describe(include='all')

,CMPLNT_FR_TM,LAW_CAT_CD,OFNS_DESC,SUSP_SEX,SUSP_RACE,VIC_SEX,VIC_RACE
count,1.935011e+06,1.935028e+06,1930917,950307,983580,1934921,1934921
unique,NaN,NaN,64,3,8,4,8
top,NaN,NaN,PETIT LARCENY,M,BLACK,F,BLACK
freq,NaN,NaN,297824,608151,469229,785631,654766
mean,5.510063e+00,2.196449e+00,NaN,NaN,NaN,NaN,NaN
std,2.199801e+00,6.423568e-01,NaN,NaN,NaN,NaN,NaN
min,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,4.000000e+00,2.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,5.000000e+00,2.000000e+00,NaN,NaN,NaN,NaN,NaN
75%,7.000000e+00,3.000000e+00,NaN,NaN,NaN,NaN,NaN


conditions = [
    (cf['SUSP_SEX'] == 'F') ,
    (cf['SUSP_SEX'] == 'M')]
choices = [1, 0]
cf['SUSP_SEX'] = np.select(conditions, choices, default=np.nan)

In [ ]:
pd.crosstab(index=cf['SUSP_SEX'], columns='count')

col_0,count
SUSP_SEX,
0.0,608151
1.0,196978


In [ ]:
conditions = [
    (cf['SUSP_RACE'] != ('BLACK' or 'BLACK HISPANIC')) ,
    (cf['SUSP_RACE'] == ('BLACK' or 'BLACK HISPANIC'))]
choices = [0, 1]
cf['SUSP_RACE'] = np.select(conditions, choices, default=np.nan)


In [ ]:
pd.crosstab(index=cf['SUSP_RACE'], columns='count')

col_0,count
SUSP_RACE,
AMERICAN INDIAN/ALASKAN NATIVE,2402
ASIAN / PACIFIC ISLANDER,19670
BLACK,469229
BLACK HISPANIC,30089
OTHER,1
UNKNOWN,231584
WHITE,112747
WHITE HISPANIC,117858


In [ ]:
pd.crosstab(index=cf['SUSP_RACE'], columns='count')

col_0,count
SUSP_RACE,
0.0,1465799
1.0,469229


In [ ]:
conditions = [
    (cf['VIC_RACE'] != ('BLACK' or 'BLACK HISPANIC')) ,
    (cf['VIC_RACE'] == ('BLACK' or 'BLACK HISPANIC'))]
choices = [0, 1]
cf['VIC_RACE'] = np.select(conditions, choices, default=np.nan)

In [ ]:
pd.crosstab(index=cf['VIC_RACE'], columns='count')

col_0,count
VIC_RACE,
0.0,1280262
1.0,654766


In [ ]:
conditions = [
    (cf['VIC_SEX'] == 'F') ,
    (cf['VIC_SEX'] == 'M')]
choices = [1, 0]
cf['VIC_SEX'] = np.select(conditions, choices, default=np.nan)

In [ ]:
pd.crosstab(index=cf['VIC_SEX'], columns='count')

col_0,count
VIC_SEX,
0.0,659932
1.0,785631


In [ ]:
lm1 = smf.ols(formula = 'LAW_CAT_CD ~ CMPLNT_FR_TM + SUSP_RACE + VIC_RACE + VIC_SEX + SUSP_SEX', data = cf).fit()
print (lm1.summary())

                            OLS Regression Results                            
Dep. Variable:             LAW_CAT_CD   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     7444.
Date:                Mon, 16 Dec 2019   Prob (F-statistic):               0.00
Time:                        22:21:23   Log-Likelihood:            -7.6704e+05
No. Observations:              691114   AIC:                         1.534e+06
Df Residuals:                  691108   BIC:                         1.534e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.9531      0.003    681.039   

In [ ]:
conditions = [
    (cf['LAW_CAT_CD'] == 1.0) ,
    (cf['LAW_CAT_CD'] > 1.0)]
choices = [0, 1]
cf['LAW_CAT_CD'] = np.select(conditions, choices, default=np.nan)

In [ ]:
pd.crosstab(index=cf['LAW_CAT_CD'], columns='count')

col_0,count
LAW_CAT_CD,
0.0,246489
1.0,1688539


In [ ]:
logit1 = sm.formula.logit(formula = "LAW_CAT_CD ~ CMPLNT_FR_TM + SUSP_RACE + VIC_RACE + VIC_SEX + SUSP_SEX", data = cf).fit()
print (logit1.summary()) 

Optimization terminated successfully.
         Current function value: 0.601770
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:             LAW_CAT_CD   No. Observations:               691114
Model:                          Logit   Df Residuals:                   691108
Method:                           MLE   Df Model:                            5
Date:                Mon, 16 Dec 2019   Pseudo R-squ.:                 0.02064
Time:                        22:47:15   Log-Likelihood:            -4.1589e+05
converged:                       True   LL-Null:                   -4.2466e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.7544      0.009     88.021      0.000       0.738       0.771
CMPLNT_FR_TM     0.0683